In [9]:
from bus_speeding import parse_data, get_speeding_buses, generate_map
from punctuality import calculate_delays

In [10]:
filepath_morning = "../data/bus-locations-2024-02-19 09:35:12.222315.json"
filepath_evening = "../data/bus-locations-2024-02-18 20:29:31.691732.json"

In [11]:
bus_to_data = parse_data(filepath_evening)

buses_speeding = get_speeding_buses(bus_to_data)

generate_map(buses_speeding)

skipped 481 elements out of 83585
found 259 buses that exceeded           speed limit out of 1769 buses


In [ ]:
delayed_buses = calculate_delays(filepath_morning)